# Kriging Example 1¶

Author: Mohit S. Chauhan Date: Jan 08, 2019

In this example, Kriging is used to generate a surrogate model for a given data. In this data, sample points are generated using STS class and functional value at sample points are estimated using python function defined in python script ('python_model_function.py).

Import the necessary libraries. Here we import standard libraries such as numpy and matplotlib, but also need to import the STS, RunModel and Krig class from UQpy.

In [1]:
from UQpy.Surrogates import Krig
from UQpy.SampleMethods import STS
from UQpy.RunModel import RunModel
import numpy as np
import matplotlib.pyplot as plt

Using UQpy STS class to generate samples for two random variables, which are uniformly distributed between 0 and 1.

In [2]:
nsamples = 20
x = STS(dimension=1, dist_name=['Gamma'], dist_params=[[2, 1, 3]], sts_design=[nsamples])

UQpy: Successful execution of STS design..


RunModel is used to evaluate function values at sample points. Model is defined as a function in python file 'python_model_function.py'. 

In [3]:
y_val = RunModel(x.samples, model_script='python_model_1Dfunction.py')

Using UQpy Krig class to generate a surrogate for generated data. In this illustration, Linear regression model and Gaussian correlation model are used.

In [4]:
K = Krig(samples=x.samples, values=np.array(y_val.qoi_list)[:,:,0], reg_model='Linear', corr_model='Gaussian', n_opt=10)

UQpy: Performing Krig...
Done!


Kriging surrogate is used to compute the response surface and its gradient.

In [5]:
num = 1000
x1 = np.linspace(min(x.samples), max(x.samples), num)
y, mse = K.interpolate(x1.reshape([num, 1]), dy=True)
y_grad = K.jacobian(x1.reshape([num, 1]))

In [6]:
Actual model is evaluated at all points to compare it with kriging surrogate.

SyntaxError: invalid syntax (<ipython-input-6-2fa3d3a7900c>, line 1)

In [ ]:
y_act = RunModel(x1, model_script='python_model_1Dfunction.py')

This plot shows the input data as blue dot, blue curve is actual function and orange curve represents response curve. This plot also shows the gradient and 95% confidence interval of the kriging surrogate.

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.plot(x1, np.array(y_act.qoi_list), label='Sine')
plt.plot(x1, y, label='Surrogate')
plt.plot(x1, y_grad, label='Gradient')
plt.scatter(K.samples, K.values, label='Data')
plt.fill(np.concatenate([x1, x1[::-1]]),
         np.concatenate([y - 1.9600 * mse,
                        (y + 1.9600 * mse)[::-1]]),
         alpha=.5, fc='y', ec='None', label='95% CI')
#plt.legend(loc='lower right')
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()